In [3]:
from dotenv import load_dotenv
import os

load_dotenv()
tavily_api_key = os.environ["TAVILY_API_KEY"]

In [4]:
from openai import OpenAI

client = OpenAI()

tavily_base_url = "https://mcp.tavily.com/mcp"
full_url = f"{tavily_base_url}/?tavilyApiKey={tavily_api_key}"

resp = client.responses.create(
    model="gpt-4.1",
    tools=[
        {
            "type": "mcp",
            "server_label": "tavily",
            "server_url": full_url,
            "require_approval": "never",
        },
    ],
    input="Do you have access to the tavily mcp server?",
)

print(resp.output_text)

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

ChatCompletion(id='chatcmpl-953', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage)], created=1769215009, model='qwen3:1.7b', object='chat.completion', service_tier=None, system_fingerprint='fp_ollama', usage=CompletionUsage(completion_tokens=630, prompt_tokens=35, total_tokens=665, completion_tokens_details=None, prompt_tokens_details=None))